#Install Requirements

In [ ]:
!pip install -U langgraph langchain langchain_openai tavily-python langchain_community chromadb

In [ ]:
!pip install "codeinterpreterapi[all]"

In [ ]:
from codeinterpreterapi import CodeInterpreterSession, settings # validation error가 뜬다면 커널을 재시작 한 후 기다렸다가 다시 시도

In [ ]:
!pip install --upgrade langchain langgraph langchain-openai pydot graphviz
import os, getpass, warnings; warnings.filterwarnings("ignore")
from langchain.prompts import PromptTemplate

In [ ]:
!pip install langchain_huggingface

In [ ]:
!huggingface-cli login

In [ ]:
from langchain_huggingface import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers.string import StrOutputParser

# 1) LLM 설정
llm = HuggingFacePipeline.from_model_id(
    model_id="google/flan-t5-base",
    task="text2text-generation",
    pipeline_kwargs={
        "max_new_tokens": 256,
        "temperature": 0.7,
    },
    device_map="auto",  # GPU가 없으면 자동으로 CPU 사용
)

In [ ]:
import os
import getpass
if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API 키를 입력하세요: ")

In [ ]:
import os
from langchain_huggingface import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 1. 모델 불러오기
llm = HuggingFacePipeline.from_model_id(
    model_id="google/flan-t5-base",
    task="text2text-generation",
    pipeline_kwargs={
        "max_new_tokens": 512,
        "temperature": 0.7,
    },
    device_map="auto",  # GPU 사용 가능 시 자동 사용
)

# 2. 프롬프트 템플릿 정의
prompt = PromptTemplate.from_template(
    """
You are an expert data analyst. Based on your knowledge, answer the following analysis request.

Request:
{user_query}

Please provide an insightful response in fluent English. Use bullet points and clear structure where applicable.
"""
)

# 3. 체인 구성
from langchain_core.runnables import RunnablePassthrough

chain = (
    {"user_query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 4. 실행 함수
def analyze_with_huggingface_model(user_query: str):
    print("📊 사용자 요청에 대한 분석을 시작합니다...\n")
    result = chain.invoke(user_query)
    print("✅ 분석 완료\n")
    print("=" * 60)
    print(result)
    print("=" * 60)

if __name__ == "__main__":
    my_request = (
        "Analyze and visualize the global AI in healthcare market size and growth forecast from 2024 to 2030. "
        "Create a bar chart comparing the market size of key regions like North America, Europe, and Asia-Pacific for the year 2024."
    )
    analyze_with_huggingface_model(my_request)


In [ ]:
import os
import getpass
import requests
import matplotlib.pyplot as plt
from langchain_huggingface import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# --- 3. Tavily 웹 검색 함수 ---
def search_tavily(query: str, max_results: int = 3):
    url = "https://api.tavily.com/search"
    headers = {"Content-Type": "application/json"}
    payload = {
        "api_key": os.environ["TAVILY_API_KEY"],
        "query": query,
        "search_depth": "basic",
        "include_answer": True,
        "max_results": max_results,
    }

    response = requests.post(url, headers=headers, json=payload)
    data = response.json()

    if "results" in data:
        return [res["content"] for res in data["results"]]
    else:
        print("검색 실패:", data)
        return []

# --- 4. 분석 및 시각화 수행 함수 ---
def analyze_and_visualize_with_huggingface(user_query: str, save_dir: str = "./figures_hf"):
    print("🔎 Tavily를 통해 정보 검색 중...")
    search_results = search_tavily(user_query)

    if not search_results:
        print("❌ 검색 결과가 없습니다.")
        return

    combined_context = "\n\n".join(search_results[:3])

    # 프롬프트 구성
    prompt = PromptTemplate.from_template("""
    You are a skilled data analyst. Use the following search results to answer the user's query with insight and clarity.

    Search Context:
    {context}

    User Query:
    {query}

    Please:
    - Provide a detailed summary and analysis.
    - Then, describe a bar chart to compare market sizes by region for 2024: North America, Europe, and Asia-Pacific.
    - Use approximate numbers if exact ones are not available.

    Output format:
    1. Textual analysis
    2. Suggested bar chart data in format: Region: Value
    """)


    # 체인 구성
    chain = (
        {"context": lambda x: combined_context, "query": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    print("\n🤖 LLM 분석 시작...\n")
    result_text = chain.invoke(user_query)
    print("✅ 분석 결과:\n")
    print("=" * 60)
    print(result_text)
    print("=" * 60)

    # 예시: 키워드 기반 시각화 조건 체크
    if "bar chart" in result_text.lower():
        print("\n📊 바 차트 생성 중...")

        # 예시 데이터 (실제 데이터 파싱 또는 사용자 요청 기반으로 수정 가능)
        labels = ['North America', 'Europe', 'Asia-Pacific']
        values = [15.2, 12.3, 18.9]

        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, "ai_healthcare_market_2024.png")

        # 시각화 생성
        plt.figure(figsize=(8, 5))
        plt.bar(labels, values, color='skyblue')
        plt.title("AI in Healthcare Market Size (2024, Example)")
        plt.ylabel("Market Size (in Billion USD)")
        plt.tight_layout()
        plt.savefig(save_path)
        plt.show()  # 👉 이 줄 추가로 화면 출력
        plt.close()

    else:
        print("📝 시각화 명령이 없거나 조건이 맞지 않습니다.")

# --- 5. 실행 예시 ---
if __name__ == "__main__":
    my_request = (
        "Analyze and visualize the global AI in healthcare market size and growth forecast from 2024 to 2030. "
        "Create a bar chart comparing the market size of key regions like North America, Europe, and Asia-Pacific for the year 2024."
    )
    analyze_and_visualize_with_huggingface(my_request)


In [ ]:
!pip install selenium
!apt-get update

# (최초 1회)
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver '/content/drive/MyDrive/Colab Notebooks' #
!pip install chromedriver-autoinstaller

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import sys
from selenium.webdriver.common.keys import Keys
import urllib.request
import os
from urllib.request import urlretrieve

import time
import pandas as pd
import numpy as np
import chromedriver_autoinstaller  # setup chrome options

In [ ]:
chrome_path = "/content/drive/MyDrive/Colab Notebooks/chromedriver"

sys.path.insert(0,chrome_path)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')  # set path to chromedriver as per your configuration
chrome_options.add_argument('lang=ko_KR') # 한국어
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

chromedriver_autoinstaller.install()  # set the target URL

In [ ]:

import re

re.findall("\d.*원", "할인가 4,920원 2%")

In [ ]:
import requests
from bs4 import BeautifulSoup as bs


keyword = "생활/주방"
url = f'https://www.gmarket.co.kr/n/best?groupCode={keyword}'#주소
res = requests.get(url)#url 주소에서 정보 가져옴
soup = bs(res.text, 'html.parser')#정보에서 text정보 parsing

title = [i.text for i in soup.find_all('p', class_='box__item-title')]

#selenium driver 로드
driver = webdriver.Chrome(options=chrome_options)

#상품 정보를 받을 리스트
prod_info_list = []

keyword = "생활/주방"
#링크 전달
driver.get(f"https://www.gmarket.co.kr/n/best?groupCode={keyword}")

#CSS_SELECTOR를 활용해서 li 태그를 모두 가져오기
lis = driver.find_elements(By.CSS_SELECTOR, "li")

#li들을 돌면서
for li in lis:
  try: #box__item-title 가져오기 시도하는데
    itemname = li.find_element(By.CSS_SELECTOR, "p.box__item-title")
  except: #실패할 경우
    continue #다음 루프로 넘어감

  #sale_price 가져오기
  sale_price = li.find_element(By.CSS_SELECTOR, "div.box__price-seller").text
  try:
    original_price = li.find_element(By.CSS_SELECTOR, "div.box__price-original").text
  except:
    original_price = ""
  #prod_info_list에 쌓기
  prod_info_list.append((itemname.text, sale_price, original_price))

#driver 종료
driver.quit()

gmarket_df = pd.DataFrame(prod_info_list, columns = ['상품명', '판매가', '원가'])
def extract_number(x):
    price = re.findall("\d.*원", x) #숫자로 시작해서 원으로 끝나는 패턴을 찾기
    #데이터가 빈 경우가 있으므로 try, except문 사용
    try:
      price_num = price[0].replace("원", "") #replace로 원을 제거
      #천원 아래 가격도 있을 수 있으므로 try, except문 사용
      try:
        price_num = int(price_num.replace(",", "")) #replace로 쉼표까지 제거하고 int로 변환
      except:
        price_num = int(price_num) #아닐 경우 그냥 int로 변환
    except:
      pass
    return price_num if price else None #데이터가 있을 경우 price_num을 반환, 아닐 경우 None을 반환

gmarket_df['원가'] = gmarket_df['원가'].apply(extract_number)

gmarket_df['판매가'] = gmarket_df['판매가'].apply(extract_number)
#각 행의 정가 열이 na면 판매가를 대입
def fill_na_with_sales(row):
    if pd.isna(row['원가']):
        return row['판매가']
    else:
        return row['원가']

gmarket_df['원가'] = gmarket_df.apply(fill_na_with_sales, axis=1)
gmarket_df['원가'] = gmarket_df['원가'].apply(int)

In [ ]:
gmarket_df.info()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from typing import TypedDict
from langgraph.graph import StateGraph, END

# 샘플 G마켓 데이터
data = gmarket_df

# State 정의
class State(TypedDict):
    query: str
    result: str

# Tool 1: 가격 분석
def price_analysis(query):
    avg_price = data['판매가'].mean()
    discount = ((data['원가'] - data['판매가']) / data['원가'] * 100).mean()
    return f"평균 가격: {avg_price:,.0f}원, 평균 할인율: {discount:.1f}%"

# Tool 2: 상품 추천
def recommend_products(query):
    best_deal = data.loc[((data['원가'] - data['판매가']) / data['원가'] * 100).idxmax()]
    return f"추천상품: {best_deal['상품명']} (가격: {best_deal['판매가']:,}원)"

# Tool 3: 시각화
def create_chart(query):
    plt.figure(figsize=(8, 5))
    plt.bar(range(len(data)), data['판매가'])
    plt.title('Product Prices')
    plt.savefig('chart.png')
    plt.close()
    return "차트 저장됨: chart.png"

# Tool 4: 검색 (더미)
def web_search(query):
    return "주방용품 시장은 성장 중입니다."

# 노드 함수들
def analyzer(state: State) -> State:
    result1 = price_analysis(state["query"])
    result2 = recommend_products(state["query"])
    result3 = create_chart(state["query"])
    result4 = web_search(state["query"])

    state["result"] = f"{result1}\n{result2}\n{result3}\n{result4}"
    return state

def output(state: State) -> State:
    print("=== 분석 결과 ===")
    print(state["result"])
    return state

# 그래프 생성
def create_agent():
    workflow = StateGraph(State)
    workflow.add_node("analyze", analyzer)
    workflow.add_node("output", output)

    workflow.set_entry_point("analyze")
    workflow.add_edge("analyze", "output")
    workflow.add_edge("output", END)

    return workflow.compile()

# 실행
def run_agent(query):
    agent = create_agent()
    result = agent.invoke({"query": query, "result": ""})
    return result

# 그래프 시각화 (텍스트)
def show_graph():
    print("Graph Structure:")
    print("User Query -> Analyzer -> Output -> End")
    print("Tools: 1.Price Analysis 2.Recommendation 3.Chart 4.Search")

# 장점/단점 분석
def analyze_benefits():
    print("\n=== 기존 Chatbot 대비 장점 ===")
    print("1. 4개 도구 동시 활용으로 종합 분석")
    print("2. 구조화된 워크플로우로 일관된 결과")
    print("3. 시각화 자동 생성")

    print("\n=== 문제점 ===")
    print("1. 모든 도구를 항상 실행하여 비효율적")
    print("2. 단계별 오류 전파 가능성")
    print("3. 외부 API 의존성")

if __name__ == "__main__":
    show_graph()
    analyze_benefits()
    print("\n=== 실행 예시 ===")
    run_agent("주방용품 추천해주세요")

#과제

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langchain_huggingface import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import requests
import os

# G마켓 샘플 데이터


# HuggingFace LLM 설정
llm = HuggingFacePipeline.from_model_id(
    model_id="google/flan-t5-base",
    task="text2text-generation",
    pipeline_kwargs={
        "max_new_tokens": 512,
        "temperature": 0.7,
    },
    device_map="auto",
)

# State 정의
class AgentState(TypedDict):
    query: str
    analysis_result: str
    search_result: str
    chart_result: str
    recommendation: str
    final_answer: str

# Tool 1: 데이터 분석
def analyze_data(state: AgentState) -> AgentState:
    # 할인율 계산
    discount_rate = ((gmarket_df['원가'] - gmarket_df['판매가']) / gmarket_df['원가'] * 100)
    avg_discount = discount_rate.mean()
    max_discount_idx = discount_rate.idxmax()
    best_deal = gmarket_df.loc[max_discount_idx]

    analysis = f"""
    상품 분석 결과:
    - 총 상품 수: {len(gmarket_df)}개
    - 평균 판매가: {gmarket_df['판매가'].mean():,.0f}원
    - 평균 할인율: {avg_discount:.1f}%
    - 최고 할인 상품: {best_deal['상품명']} ({discount_rate[max_discount_idx]:.1f}% 할인)
    """

    state["analysis_result"] = analysis
    return state

# Tool 2: 웹 검색 (Tavily API 또는 더미)
def web_search(state: AgentState) -> AgentState:
    # Tavily API 키가 있으면 실제 검색, 없으면 더미 데이터
    if "TAVILY_API_KEY" in os.environ:
        try:
            url = "https://api.tavily.com/search"
            headers = {"Content-Type": "application/json"}
            payload = {
                "api_key": os.environ["TAVILY_API_KEY"],
                "query": f"kitchen appliances market trends Korea {state['query']}",
                "search_depth": "basic",
                "max_results": 2,
            }
            response = requests.post(url, headers=headers, json=payload)
            data = response.json()
            if "results" in data:
                search_info = "\n".join([res["content"][:100] for res in data["results"]])
            else:
                search_info = "주방용품 시장은 지속적으로 성장하고 있습니다."
        except:
            search_info = "주방용품 시장은 지속적으로 성장하고 있습니다."
    else:
        search_info = "주방용품 시장은 지속적으로 성장하고 있습니다. 스테인리스 제품이 인기가 높습니다."

    state["search_result"] = search_info
    return state

# Tool 3: 시각화
def create_visualization(state: AgentState) -> AgentState:
    # 가격 분포 차트 생성
    plt.figure(figsize=(10, 6))
    plt.subplot(1, 2, 1)
    plt.bar(range(len(gmarket_df)), gmarket_df['판매가'], alpha=0.7)
    plt.title('Product Sale Prices')
    plt.xlabel('Products')
    plt.ylabel('Price (KRW)')

    # 할인율 차트
    plt.subplot(1, 2, 2)
    discount_rate = ((gmarket_df['원가'] - gmarket_df['판매가']) / gmarket_df['원가'] * 100)
    plt.bar(range(len(gmarket_df)), discount_rate, alpha=0.7, color='green')
    plt.title('Discount Rates')
    plt.xlabel('Products')
    plt.ylabel('Discount Rate (%)')

    plt.tight_layout()
    plt.show()
    plt.savefig('gmarket_analysis.png', dpi=150, bbox_inches='tight')
    plt.close()

    state["chart_result"] = "시각화 차트가 'gmarket_analysis.png'로 저장되었습니다."
    return state

# Tool 4: 추천 시스템
def recommend_products(state: AgentState) -> AgentState:
    # 할인율 기준 상위 3개 추천
    discount_rate = ((gmarket_df['원가'] - gmarket_df['판매가']) / gmarket_df['원가'] * 100)
    gmarket_df['할인율'] = discount_rate
    top_deals = gmarket_df.nlargest(3, '할인율')

    recommendations = "추천 상품 (할인율 기준):\n"
    for idx, row in top_deals.iterrows():
        recommendations += f"- {row['상품명']}: {row['판매가']:,}원 ({row['할인율']:.1f}% 할인)\n"

    state["recommendation"] = recommendations
    return state

# LLM을 사용한 최종 분석
def llm_synthesizer(state: AgentState) -> AgentState:
    # 프롬프트 템플릿
    prompt = PromptTemplate.from_template("""
    You are an expert shopping advisor. Based on the following analysis results, provide comprehensive shopping advice in Korean.

    User Query: {query}
    Data Analysis: {analysis}
    Market Search: {search}
    Chart Info: {chart}
    Recommendations: {recommendations}

    Please provide actionable shopping insights and advice in Korean. Focus on helping the user make informed purchasing decisions.
    """)

    # 체인 구성
    chain = (
        {
            "query": lambda x: state["query"],
            "analysis": lambda x: state["analysis_result"],
            "search": lambda x: state["search_result"],
            "chart": lambda x: state["chart_result"],
            "recommendations": lambda x: state["recommendation"]
        }
        | prompt
        | llm
        | StrOutputParser()
    )

    # LLM 실행
    final_response = chain.invoke({})
    state["final_answer"] = final_response
    return state

# 결과 출력
def output_results(state: AgentState) -> AgentState:
    print("=" * 50)
    print("🛒 G마켓 쇼핑 분석 결과")
    print("=" * 50)
    print("\n📊 데이터 분석:")
    print(state["analysis_result"])
    print("\n🔍 시장 검색 결과:")
    print(state["search_result"])
    print("\n📈 시각화:")
    print(state["chart_result"])
    print("\n💡 추천 상품:")
    print(state["recommendation"])
    print("\n🤖 LLM 종합 분석:")
    print(state["final_answer"])
    print("=" * 50)
    return state

# LangGraph 생성
def create_gmarket_agent():
    workflow = StateGraph(AgentState)

    # 노드 추가
    workflow.add_node("analyzer", analyze_data)
    workflow.add_node("searcher", web_search)
    workflow.add_node("visualizer", create_visualization)
    workflow.add_node("recommender", recommend_products)
    workflow.add_node("synthesizer", llm_synthesizer)
    workflow.add_node("output", output_results)

    # 엣지 설정
    workflow.set_entry_point("analyzer")
    workflow.add_edge("analyzer", "searcher")
    workflow.add_edge("searcher", "visualizer")
    workflow.add_edge("visualizer", "recommender")
    workflow.add_edge("recommender", "synthesizer")
    workflow.add_edge("synthesizer", "output")
    workflow.add_edge("output", END)

    return workflow.compile()

# 실행 함수
def run_agent(query: str):
    agent = create_gmarket_agent()

    initial_state = {
        "query": query,
        "analysis_result": "",
        "search_result": "",
        "chart_result": "",
        "recommendation": "",
        "final_answer": ""
    }

    print(f"🚀 사용자 요청: {query}")
    result = agent.invoke(initial_state)
    return result

# 그래프 시각화 (텍스트)
def show_graph_structure():
    print("\n📋 LangGraph 구조:")
    print("User Query → Data Analyzer → Web Searcher → Visualizer → Recommender → LLM Synthesizer → Output → End")
    print("\n🔧 사용된 도구들:")
    print("1. Data Analysis Tool: 가격 분석, 할인율 계산")
    print("2. Web Search Tool: Tavily API로 시장 트렌드 검색")
    print("3. Visualization Tool: matplotlib으로 차트 생성")
    print("4. Recommendation Tool: 할인율 기반 상품 추천")
    print("5. LLM Synthesizer: HuggingFace 모델로 종합 분석")

# 장점/단점 분석
def analyze_agent_performance():
    print("\n🚀 기존 Chatbot 대비 장점:")
    print("- 다단계 분석: 데이터분석→검색→시각화→추천→LLM종합의 체계적 프로세스")
    print("- 도구 조합: 5개 도구를 연계하여 종합적 쇼핑 조언 제공")
    print("- 실시간 정보: 웹 검색으로 최신 시장 트렌드 반영")
    print("- 시각적 분석: 자동 차트 생성으로 직관적 정보 제공")

    print("\n⚠️ 문제점:")
    print("- 처리 시간: 순차적 실행으로 응답 시간 증가")
    print("- 오류 전파: 한 단계 실패 시 전체 파이프라인 영향")
    print("- 복잡성: 간단한 질문에도 모든 단계 실행")
    print("- API 의존: 외부 서비스 장애 시 기능 제한")

if __name__ == "__main__":
    # 그래프 구조 출력
    show_graph_structure()

    # 성능 분석 출력
    analyze_agent_performance()

    # 에이전트 실행
    print("\n" + "="*50)
    print("🛒 G마켓 LLM Agent 실행 예시")
    print("="*50)

    # 테스트 실행
    result = run_agent("할인율이 높은 주방용품을 추천해주세요")